In [7]:
!pip install yfinance pandas

import yfinance as yf
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd

In [8]:
SECTOR_MAP = {
    'Technology': ['MSFT', 'GOOGL', 'META', 'NVDA', 'ADBE', 'INTC', 'CSCO', 'ORCL', 'AAPL'],
    'Healthcare': ['JNJ', 'PFE', 'MRK', 'ABT', 'AMGN', 'GILD', 'BMY'],
    'Financial Services': ['JPM', 'BAC', 'WFC', 'C', 'GS', 'MS', 'AXP'],
    'Consumer Cyclical': ['AMZN', 'TSLA', 'HD', 'MCD', 'NKE', 'SBUX'],
    'Industrials': ['HON', 'GE', 'BA', 'CAT', 'MMM', 'UNP'],
    'Energy': ['XOM', 'CVX', 'SHEL', 'COP', 'BP', 'EOG'],
    'Utilities': ['NEE', 'DUK', 'SO', 'D', 'AEP', 'EXC'],
    'Real Estate': ['AMT', 'PLD', 'EQIX', 'CCI', 'PSA', 'SPG']
}

ticker_input = widgets.Text(description='Ticker:')
compute_btn = widgets.Button(description='Compare Sector P/Es', button_style='success')
output = widgets.Output()

def get_sector_pe_comparison(b):
    with output:
        clear_output()
        ticker = ticker_input.value.strip().upper()

        if not ticker:
            print("Please enter a valid ticker symbol")
            return

        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            base_pe = info.get('trailingPE')
            sector = info.get('sector')

            if not base_pe:
                print(f"No P/E ratio available for {ticker}")
                return
            if not sector:
                print(f"No sector information available for {ticker}")
                return

            sector_peers = SECTOR_MAP.get(sector, [])
            if not sector_peers:
                print(f"No predefined peers available for {sector} sector")
                return

            peers_data = []
            for peer in sector_peers:
                if peer == ticker:
                    continue
                try:
                    p = yf.Ticker(peer)
                    p_info = p.info
                    p_pe = p_info.get('trailingPE')
                    if p_pe and p_pe > 0:
                        peers_data.append({
                            'Ticker': peer,
                            'Company': p_info.get('shortName', peer),
                            'P/E Ratio': p_pe
                        })
                except Exception as e:
                    continue
                if len(peers_data) >= 5:
                    break

            if not peers_data:
                print(f"No valid P/E ratios found in {sector} sector")
                return

            comparison = [{
                'Ticker': ticker,
                'Company': info.get('shortName', ticker),
                'P/E Ratio': base_pe
            }] + peers_data

            df = pd.DataFrame(comparison).sort_values('P/E Ratio', ascending=False)

            print(f"{info.get('shortName', ticker)} ({ticker}) Sector P/E Comparison")
            print(f"Sector: {sector}\n")
            print("Top Sector Comparisons:")
            display(df.style.format({'P/E Ratio': '{:.2f}'}))
            print("\nNote: Comparison uses major sector constituents. Data from Yahoo Finance")

        except Exception as e:
            print(f"Error processing {ticker}: {str(e)}")

compute_btn.on_click(get_sector_pe_comparison)
input_box = widgets.HBox([ticker_input, compute_btn])
main_layout = widgets.VBox([input_box, output], layout=widgets.Layout(padding='20px'))

print("Enter a stock ticker to compare its P/E ratio with sector peers:")
display(main_layout)

Enter a stock ticker to compare its P/E ratio with sector peers:
